# Azure Storage Service Demo
This is intended to be run on the Python 2 kernel.   
If you have not installed azure and azure-storage un-comment the pip below and run it.

In [1]:
!pip install --upgrade azure-storage==0.32.0

     |████████████████████████████████| 160 kB 8.2 MB/s eta 0:00:01


In [2]:
import csv
import sys
import azure.storage
from azure.storage.table import TableService, Entity
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess
import time
print("hello at" + time.asctime(time.localtime(time.time())))

hello atMon Feb 21 12:40:00 2022


follow the instructions in the book to create a storage account.   the one used here is "tutorial", but you will want to pick something else and replace that name everywhere below

In [4]:
account = 'aronlab2'
block_blob_service = BlockBlobService(account_name=account,
    account_key='tOwlO2VLZuuiOttJM8623uZVLjXsRtsmwX9L35tpi9aW8uOJ69movkuXWg86q0vXFV+KCl5edO0/+ASt3tqI1A==')
block_blob_service.create_container('datacont', 
                                    public_access=PublicAccess.Container)

True

In [5]:
table_service = TableService(account_name=account, 
                             account_key='tOwlO2VLZuuiOttJM8623uZVLjXsRtsmwX9L35tpi9aW8uOJ69movkuXWg86q0vXFV+KCl5edO0/+ASt3tqI1A==')

In [6]:
if table_service.create_table('DataTable'):
    print("table created")
else:
    print("table already there")

table created


Now you need a directory with a csv file.  For example

'experiment1', '1', '3/15/2002', 'exp1', 'this is the comment'

'experiment1', '2', '3/15/2002', 'exp2', 'this is the comment2'

'experiment2', '3', '3/16/2002', 'exp3', 'this is the comment3'

'experiment3', '4', '3/16/2002', 'exp4', 'this is the comment233'

now you need a directory datafiles that has four blobs of any type.  call them exp1, exp2, exp3, exp4.  
we have one ready for you if you are using the tutorial container. It is stored in /datadir.
If you are running this somewhere else you will need to make your own.  
let's see what is there.

In [16]:
 with open('/Users/nanohana/Downloads/Cloud_computing/assignment2/datadir/experiments.csv') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)

['experiment1', ' id1', '2020-06-28T00:11:53.155Z', 'a.jpg', ' 12/1/2016', '" ""here is a view of what is to be made"""', 'https://tutorialjcf.blob.core.windows.net/datacont/a.jpg']
['experiment1', ' id2', '2020-06-28T00:11:53.293Z', 'b.jpg', ' 12/3/2016', '" ""map rededuce picture"""', 'https://tutorialjcf.blob.core.windows.net/datacont/b.jpg']
['experiment1', ' id5', '2020-06-28T00:11:53.631Z', 'e.jpg', ' 12/6/2016', '" ""bio samples"""', 'https://tutorialjcf.blob.core.windows.net/datacont/e.jpg']
['experiment2', ' id3', '2020-06-28T00:11:53.427Z', 'c.jpg', ' 12/4/2016', '" ""sample notebook"""', 'https://tutorialjcf.blob.core.windows.net/datacont/c.jpg']
['experiment3', ' id4', '2020-06-28T00:11:53.511Z', 'd.jpg', ' 12/5/2016', '" ""workers"""', 'https://tutorialjcf.blob.core.windows.net/datacont/d.jpg']


the code below will upload the data blobs into the container "datacont", then create the url and upload the metadata into the table "DataTable".

In [17]:
 with open('/Users/nanohana/Downloads/Cloud_computing/assignment2/datadir/experiments.csv') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)
        block_blob_service.create_blob_from_path(
            'datacont', item[3],
            "/Users/nanohana/Downloads/Cloud_computing/assignment2/datadir/"+item[3]
            )
        url = "https://"+account+".blob.core.windows.net/datacont/"+item[3]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1], 
                 'description' : item[5], 'date' : item[4], 'url':url} 
        table_service.insert_entity('DataTable', metadata_item)
        

['experiment1', ' id1', '2020-06-28T00:11:53.155Z', 'a.jpg', ' 12/1/2016', '" ""here is a view of what is to be made"""', 'https://tutorialjcf.blob.core.windows.net/datacont/a.jpg']
['experiment1', ' id2', '2020-06-28T00:11:53.293Z', 'b.jpg', ' 12/3/2016', '" ""map rededuce picture"""', 'https://tutorialjcf.blob.core.windows.net/datacont/b.jpg']
['experiment1', ' id5', '2020-06-28T00:11:53.631Z', 'e.jpg', ' 12/6/2016', '" ""bio samples"""', 'https://tutorialjcf.blob.core.windows.net/datacont/e.jpg']
['experiment2', ' id3', '2020-06-28T00:11:53.427Z', 'c.jpg', ' 12/4/2016', '" ""sample notebook"""', 'https://tutorialjcf.blob.core.windows.net/datacont/c.jpg']
['experiment3', ' id4', '2020-06-28T00:11:53.511Z', 'd.jpg', ' 12/5/2016', '" ""workers"""', 'https://tutorialjcf.blob.core.windows.net/datacont/d.jpg']


Next let's do a querry for experiment1 and project onto the urls.  it should print the urls created above

In [18]:
tasks = table_service.query_entities('DataTable', filter="PartitionKey eq 'experiment1'", select='url')
for task in tasks:
    print(task.url)

https://aronlab2.blob.core.windows.net/datacont/a.jpg
https://aronlab2.blob.core.windows.net/datacont/b.jpg
https://aronlab2.blob.core.windows.net/datacont/e.jpg


The output should look somethink like

https://escistore.blob.core.windows.net/datacont/exp1

https://escistore.blob.core.windows.net/datacont/exp2

try clicking on the links in your output


Next download the "azure storage explorer" and look at you table.

You will need to add the account with the key to the storage explore. 